# The Logistic Regression

    Thomas Moreau <thomas.moreau@inria.fr>
    Alexandre Gramfort <alexandre.gramfort@inria.fr>
    
    Notebook inspired from materials by M. Le Morvan, O. Grisel.

In this notebook, we  will implement a logistic regression model in Python using only `numpy` library (and `matplotlib` for visualization), to get the basic tools necessary to implement a neural network for classification.

## Table of contents


1. [Data Generation](#data)  
2. [The Logistic Regression](#logreg)  
    2.1. [Loss and Gradient](#2.1---Loss-and-Gradient)  
    2.2. [Stochastic Gradient Descent](#2.2---Stochastic-Gradient-Descent)  
    2.3. [Scikit-learn Model](#2.3---Scikit-learn-Model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

### 1 - Data Generation

<a id='data'></a>

To test our model on very simple example, we will use Gaussian data.

In [ ]:
from utils import generate_data
from utils import plot_data, show_decision_boundary

MU1 = (2, 0)
MU2 = (-1, np.sqrt(3))
MU3 = (-1, -np.sqrt(3))

l_mu = [MU1, MU2, MU3]
X, y = generate_data(l_mu, random_state=42)

# One hot encoded target
y_ohe = OneHotEncoder().fit_transform(y[:, None]).toarray()

plot_data(X, y)

Split data into a training and a validation set.

In [ ]:
(X_train, X_val,
 y_train, y_val,
 y_ohe_train, y_ohe_val) = train_test_split(X, y, y_ohe)

## 2 - Logistic Regression

<a id='logreg'></a>


### 2.1 - The model

We considers $N$ data points $(\mathbf{x}_1, \dots, \mathbf{x}_N)$ in $\mathbb{R}^{D}$ belonging to $C$ possible classes.

We want to solve the classification task for these points, _i.e.,_ learn the parameters $\theta = (\mathbf{w}, \mathbf{b}) \in \mathbb{R}^{C\times D}\times \mathbb{R}^{C}$ of the function $f_\theta: \mathbb{R}^D \to [0, 1]^C$ which corresponds for each coordinate to the probability of being from one class. For a given $\mathbf{x}$, the model is defined as
$$
\hat{\mathbb{P}}[Y=c |\mathbf{X} = \mathbf{x}] = \frac{1}{Z} \exp(\mathbf{w}_c^\top\mathbf{x}+b_c).
$$
As these probabilities must sum to one, we get
$$
Z = \sum_{c=1}^C \exp(\mathbf{w}_c^\top\mathbf{x}+b_c).
$$
We can recognize the so-called _soft-max_ function $\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^C e^{z_j}}$.

<div class="alert alert-success">
    <b>Exercice:</b>

* Write the function that returns the probability of being of one class given the parameters `w, b` of the model and the input `X`.

</div>

**Hint:** You can use the function `scipy.special.softmax`

Solution in `solutions/0_logreg_model.py`.

In [ ]:
from scipy.special import softmax


def predict_proba_logreg(w, b, X):
    """Return the proba of being in one of two classes.

    Parameters
    ----------
    w : ndarray, shape (n_classes, n_features)
        parameters for the linear model.
    b : ndarray, shape (n_classes,)
        biases for the linear model.
    X : ndarray, shape (n_samples, n_features)
        input data

    Return
    ------
    y_proba : ndarray, shape (n_samples, n_classes)
        probability of being from each class according to
        the linear model w.
    """

    ####################
    # TO DO

    # END TO DO
    ####################
    return y_proba


The following cell shows the decision boundary of such model.

In [ ]:
w = np.array([[3, 2], [-2, 3], [5, 0]])
b = np.array([.1, -.1, .3])

y_pred = predict_proba_logreg(w, b, X_val).argmax(axis=1)
print(f"Model accuracy: {np.mean(y_pred == y_val)}")

show_decision_boundary(partial(predict_proba_logreg, w=w, b=b), data=(X, y))

### 2.2 Loss and Gradient

To learn the model parameters $\mathbf{w}_c, b_c$ on a training set $\{\mathbf{x}_i, y_i\}$, we minimize the negative log likelihood (**nll**, _a.k.a_ cross-enropy loss):
$$
L(\mathbf{w}, \mathbf{b}) = - \frac1N\sum_{i=1}^N \log(\hat{\mathbb{P}}[Y=y_i |\mathbf{X} = \mathbf{x}_i]) = -\frac1N \sum_{i=1}^N \log\Bigg(\frac{\mathbf{w}_{y_i}^\top\mathbf{x}_i+b_{y_i}}{\sum_{c=1}^C \exp(\mathbf{w}_{c}^\top\mathbf{x}_i+b_{c})}\Bigg)
$$


As we saw previously, to train a model, we will need to compute the gradients of this loss with respect to its parameters. Let's first start with the derivative of the loss relative to the prediction for one sample. If we denote $Y$ the one-hot encoded target for one sample and $\mathbf{\hat y}$ corresponds to the output of the model, the **nll** reads:
$$
\ell(Y, \mathbf{\hat y}) = - \sum_{c=1}^C Y_{c} \log(\mathbf{\hat y}_{c})
= - \sum_{c=1}^C Y_c \left(z_c - \log\left(\sum_{l=1}^C \exp(z_l)\right)\right).
$$
where $z_c = \mathbf{w}_{c}^\top\mathbf{x}_i+b_{c}$

Consequently,
$$
\frac{\partial \ell }{\partial z_j} = - \left(Y_j - \sum_{c=1}^C Y_c \frac{\exp(z_j)}{\sum_{l=1}^C \exp(z_l)} \right) = -\left(Y_j - \sum_{c=1}^C Y_c \mathbf{ \hat y}_j \right) = \mathbf{ \hat y}_j - Y_j.
$$

Note that deriving the log of the softmax makes calculations much easier.  
Now, using the chain rule, we can easily compute the gradient for $\mathbf{w}$ and $\mathbf{b}$:

$$
\begin{split}
    \frac{\partial \ell}{\partial w_j} = \frac{\partial z_j}{\partial w_j}^\top\frac{\partial \ell}{\partial z_j} = (\mathbf{\hat y}_j - Y_j)\mathbf{x}\\
    \frac{\partial \ell}{\partial b_j} = \frac{\partial z_j}{\partial b_j}^\top\frac{\partial \ell}{\partial z_j} = (\mathbf{\hat y}_j - Y_j)
\end{split}
$$

And using the linearity of the gradient, we get:

$$
\begin{split}
    \frac{\partial L}{\partial w_j} = \frac1N\sum_{i=1}^N(\mathbf{\hat y}_{ij} - Y_{ij})\mathbf{x_i}\\
    \frac{\partial L}{\partial b_j} = \frac1N\sum_{i=1}^N (\mathbf{\hat y}_{ij} - Y_{ij})
\end{split}
$$


<div class="alert alert-success">
    <b>Exercice:</b>

* Write the corresponding function which returns the log-likelihood of the model as well as its gradient.

</div>

Solution in `solutions/1_logreg_gradient.py`

In [ ]:
def log_likelihood_and_grad(w, b, X, y):
    """Log-likelihood of the logistic regression model and gradient.

    Parameters
    ----------
    w : ndarray, shape (n_classes, n_features)
        parameters for the linear model.
    b : ndarray, shape (n_classes,)
        biases for the linear model.
    X : ndarray, shape (n_samples, n_features)
        input data
    y : ndarray, shape (n_samples, n_classes)
        output targets one hot encoded.

    Returns
    -------
    loss : log-likelihood of the logreg model
    grad_w : gradient of the model parameters w.
    grad_b : gradient of the model parameters b.
    """

    #####################
    # TO DO

    # END TO DO
    #####################
    return loss, grad_w, grad_b


In [ ]:
w = np.array([[3, 2], [-2, 3], [5, 0]])
b = np.array([.1, -.1, .3])

loss, grad_w, grad_b = log_likelihood_and_grad(w, b, X_train, y_ohe_train)
print("Computed loss =", loss)
assert np.allclose(loss, 3.004595)

### 2.3 - Stochastic Gradient Descent

The stochastic gradient descent for this model consists in taking a sub part of the training data at random, computing the "stochastic" gradient and updating the parameters of the model.
The algorithm is the following:

1. Initialize the model's parameter $\mathbf{w}$ and $\mathbf{b}$ at random.
2. Iterate for a certain number of iterations:
    1. Select a sub part $B$ of the dataset $\{\mathbf{x}_i, y_i\}$ for $i$ sample at random.
    2. Compute the gradient of the loss with respect to these data points:
    $$
    \begin{split}
        \frac{\partial \widetilde L}{\partial w_j} = \frac1{|B|}\sum_{i\in B}(\mathbf{\hat y}_{ij} - Y_{ij})\mathbf{x_i}\\
        \frac{\partial \widetilde L}{\partial b_j} = \frac1{|B|}\sum_{i\in B} (\mathbf{\hat y}_{ij} - Y_{ij})
    \end{split}
    $$
    3. Update the parameters $w \leftarrow w - \eta\frac{\partial \widetilde L}{\partial w}$ and $b \leftarrow b - \eta\frac{\partial \widetilde L}{\partial b}$.
    
The core idea is that in average over $B$, $\mathbb E_B \frac{\partial \widetilde L}{\partial w} = \frac{\partial \widetilde L}{\partial w}$

<div class="alert alert-success">

**Exercice:**

Code a stochastic gradient descent using the previous function to compute the gradient of minibatch of size 128:

* Select a minibatch of `batch_size` samples in the training set. You can use the function `rng.choice` for this.
* Compute the loss and gradient.
* Update the parameters of the model `w` and `b` with a step size `lr`.
* Compute the validation loss as `val_loss`.
    
_Note that a more complete loop would make sure each training sample is visited on each epochs. Here we simplify the loop to make it easier to read._

</div>

Solution in `solutions/2_logreg_sgd.py`.

In [ ]:
pobj = []
l_predict_proba = []


def logger(i, w, b, loss):
    pobj.append(loss)
    l_predict_proba.append(
        partial(predict_proba_logreg, w=w.copy(), b=b.copy())
    )

    if i % 100 == 0:
        y_pred = predict_proba_logreg(w, b, X_val).argmax(axis=1)
        print(f"Iterarion {i} - validation accuracy: {np.mean(y_pred == y_val)}")

# Initialize the weights of the model
w = np.array([[3., 2], [-2., 3.], [5., 0]])
b = np.array([.1, -.1, .3])

# Parameters of the SGD
lr = 2e-2  # learning rate
batch_size = 16
patience = 100

# Constants for the early stopping
best_iter = 0
best_loss_val = 1e100
best_params = (w.copy(), b.copy())


n_samples, n_features = X_train.shape
rng = np.random.RandomState(72)

for it in range(5000):
    ##############################
    # TODO

    # END TODO
    ##########################

    # Logger to monitor the progress
    # of the training.
    logger(it, w, b, loss)

    # Early stopping mechanism:
    # - store the best loss and params
    # - stop if no progress after patience iterations
    if best_loss_val > val_loss:
        best_iter = it
        best_loss_val = val_loss
        best_params = (w.copy(), b.copy())

    if it - best_iter >= patience:
        print(
            "Stopping as no progress has been made "
            f"for {patience} iterations"
        )
        w, b = best_params
        break

plt.plot(pobj)

y_pred = predict_proba_logreg(w, b, X_val).argmax(axis=1)
print(f"Final validation accuracy: {np.mean(y_pred == y_val)}")

show_decision_boundary(partial(predict_proba_logreg, w=w, b=b), data=(X, y))


In [ ]:
from utils import create_animation
create_animation(l_predict_proba, X, y, iter_step=25)